# 라이브러리 설치 및 import

In [1]:
pip install numpy scipy matplotlib opencv-python

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, ifft2, fftshift, ifftshift

# 이미지 경로

In [22]:
# 경로
image_folder = ""
output_folder = ""

# FFT

In [23]:
def apply_fft(image):
    f_transform = fft2(image)
    f_transform_shifted = fftshift(f_transform)
    magnitude_spectrum = 20 * np.log(np.abs(f_transform_shifted) + 1)
    return f_transform_shifted, magnitude_spectrum

# Notch Filter적용

## 노이즈 중심 좌표 자동 탐지

In [24]:
def find_noise_centers(magnitude_spectrum, threshold_factor=2):
    mean_val = np.mean(magnitude_spectrum)
    threshold = threshold_factor * mean_val
    noise_positions = np.where(magnitude_spectrum > threshold)

    rows, cols = magnitude_spectrum.shape
    crow, ccol = rows // 2, cols // 2

    centers = []
    for r, c in zip(noise_positions[0], noise_positions[1]):
        if abs(r - crow) > 10 or abs(c - ccol) > 10:
            centers.append((c, r))
    
    return centers

## 다중 Notch Filter 적용함수

In [25]:
def apply_multi_notch_filter(f_transform_shifted, centers, notch_radius=20):
    rows, cols = f_transform_shifted.shape
    notch_filter = np.ones((rows, cols), np.uint8)
    
    for center in centers:
        cv2.circle(notch_filter, center, notch_radius, 0, thickness=-1)
    
    filtered_transform_shifted = f_transform_shifted * notch_filter
    return filtered_transform_shifted

# IFFT

In [26]:
def apply_ifft(filtered_transform_shifted):
    f_transform_filtered = ifftshift(filtered_transform_shifted)
    image_filtered = ifft2(f_transform_filtered)
    image_filtered = np.abs(image_filtered)
    return image_filtered

# 컬러이미지 처리

In [29]:
def process_color_image(image, notch_radius=20, threshold_factor=2):
    # BGR 채널 분리
    channels = cv2.split(image)
    
    filtered_channels = []
    
    for channel in channels:
        # FFT 적용
        f_transform_shifted, magnitude_spectrum = apply_fft(channel)
        
        # 노이즈 중심 좌표 자동 탐지
        noise_centers = find_noise_centers(magnitude_spectrum, threshold_factor)
        
        # 다중 Notch Filter 적용
        filtered_transform_shifted = apply_multi_notch_filter(f_transform_shifted, noise_centers, notch_radius)
        
        # Inverse FFT 적용
        filtered_channel = apply_ifft(filtered_transform_shifted)
        filtered_channels.append(filtered_channel)
    
    # 필터링된 채널 합치기 (BGR 순서로)
    filtered_image = cv2.merge(filtered_channels)
    
    return filtered_image

# 실행

In [39]:
def process_all_images_in_folder(folder_path, output_folder, notch_radius=20, threshold_factor=2):
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            file_path = os.path.join(folder_path, filename)
            
            # 이미지 불러오기 (컬러로 불러오기)
            image = cv2.imread(file_path)
            
            filtered_image = process_color_image(image, notch_radius, threshold_factor)
            
            # 결과 이미지 저장
            filtered_image_path = os.path.join(output_folder, f"filtered_{filename}")
            cv2.imwrite(filtered_image_path, filtered_image.astype(np.uint8))  # 필터링 결과 저장


process_all_images_in_folder(image_folder, output_folder, notch_radius=15, threshold_factor=2.5)